In [11]:
import pandas
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [12]:
df = pandas.read_csv('./analysis_df.csv')
df_student = df[df['Title'] == 'Student']
df_employee = pandas.read_csv('./analysis_df_employee.csv')

-----
### plot 1

In [183]:
# Get every platfroms from everyone choice
learning_platfroms_employee = df_employee[['Learning Platforms']].apply(lambda x: 
                                                                        list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                                                .explode().value_counts().to_frame(name='Vote').drop(index = 'nan')
learning_platfroms = df_student[['Learning Platforms']].apply(lambda x: 
                                                      list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                                        .explode().value_counts().to_frame(name='Vote').drop(index = 'nan')

In [163]:
# --- drop un useful records
for i in learning_platfroms.index:
    if i.isnumeric():
        learning_platfroms.drop(index = i,inplace=True)
learning_platfroms.drop(index = '-1',inplace=True)

for i in learning_platfroms_employee.index:
    if i.isnumeric():
        learning_platfroms_employee.drop(index = i,inplace=True)
        
learning_platfroms_employee.drop(index = '-1',inplace=True)

# -----

learning_platfroms.reset_index(inplace=True)
learning_platfroms.rename(columns={'index':'Platfrom'},inplace=True)

learning_platfroms_employee.reset_index(inplace=True)
learning_platfroms_employee.rename(columns={'index':'Platfrom'},inplace=True)


In [164]:
# get top 15
learning_platfroms=learning_platfroms.sort_values('Vote',ascending=False).head(15)
learning_platfroms_employee=learning_platfroms_employee.sort_values('Vote',ascending=False).head(15)

In [165]:
fig = make_subplots(rows=1,cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=("Student", "Employee"))
fig.add_trace(go.Pie(
     name='',
     values=learning_platfroms['Vote'],
     labels=learning_platfroms['Platfrom'],hole=0.3),  
     row=1, col=1)

fig.add_trace(go.Pie(
     name='',
     values=learning_platfroms_employee['Vote'],
     labels=learning_platfroms_employee['Platfrom'],hole=0.3),
    row=1, col=2)

fig.update_layout(height=600, width=1600, title_text="Learning Platforms")

Nhận xét: Với 15 nền tảng khảo sát được chọn nhiều nhất bởi 2 phía đều được giữa nguyên mà không có sự xuất hiện của một nền tảng nào khác đối với bên còn lại, dẫn đầu bởi `Coursera` và `Udemy` đã chiếm một phần phổ biến rất lớn đối với nhưng người được khảo sát khi có đến khoảng 1/3 số người lựa chọn nền tảng này, cho thấy rằng không có sự khác biệt lớn giữa những người đi học về đã đi làm ở khoảng mục này

--- 
### plot 2

In [173]:
# Get every Language from everyone choice
Languages_employee = df_employee[['Languages']].apply(lambda x: 
                                                            list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                                    .explode().value_counts().to_frame(name='Employee')
Languages = df_student[['Languages']].apply(lambda x: 
                                            list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                            .explode().value_counts().to_frame(name='Student')

In [174]:
# --- drop un useful records
for i in Languages_employee.index:
    if i.isnumeric():
        Languages_employee.drop(index = i,inplace=True)
Languages_employee.drop(index = ['-1','nan'],inplace=True)


for i in Languages.index:
    if i.isnumeric():
        Languages.drop(index = i,inplace=True)
        
Languages.drop(index = ['-1','nan'],inplace=True)

# -----

Languages.reset_index(inplace=True)
Languages.rename(columns={'index':'Languages'},inplace=True)

Languages_employee.reset_index(inplace=True)
Languages_employee.rename(columns={'index':'Languages'},inplace=True)
Languages_use= pandas.merge(Languages,Languages_employee,how='inner')

Languages_use_percent  = Languages_use
Languages_use_percent[['Student','Employee']] =  Languages_use[['Student','Employee']]/Languages_use[['Student','Employee']].sum()

In [175]:
plot = go.Figure(data=[go.Bar(
    name = 'Student',
    x = Languages_use_percent['Languages'],
    y = Languages_use_percent['Student'],
   ),
                       go.Bar(
    name = 'Employee',
    x = Languages_use_percent['Languages'],
    y = Languages_use_percent['Employee']
   )
])  
plot.update_layout(
    xaxis_title="Languages", yaxis_title="Percentage"
)               
plot.show()

Nhận xét: Không lạ khi Python vẫn là lựa chọn hàng đầu của lĩnh vực này, và cũng không có nhiều khác biệt lớn giữa người đi làm và người chưa đi làm, ngoại trừ việc người đi làm ta thấy nhỉnh hơn về số lượng các ngôn ngữ mang chuyên tính chuyên môn 'khá hơn' như R, Bash, SQL, Scala, VBA, ... trong khi những người chưa đi làm thì có xu thế học những ngôn ngữ mang tính 'đề cử' cho người bắt đầu học như java, C hay C++

----
### Plot 3

In [187]:
employee_paper = df_employee['Published Papers'].value_counts().to_frame()
not_employee_paper = df['Published Papers'].value_counts().to_frame() - employee_paper

In [199]:
fig = make_subplots(rows=1,cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=("Employee","Unemployment"))
fig.add_trace(go.Pie(
     name='',
     values=employee_paper['Published Papers'],
     labels=employee_paper.index.tolist(),hole=0.3),  
     row=1, col=1)

fig.add_trace(go.Pie(
     name='',
     values=not_employee_paper['Published Papers'],
     labels=not_employee_paper.index.tolist(),hole=0.3),
    row=1, col=2)

fig.update_layout(height=600, width=1000, title_text="Published Papers")

nhìn chung ta có thể thấy được việc có bào báo publish hay không cũng có một phần ảnh hưởng đến khả năng có việc của những người khảo sát khi đa phần những người chưa có việc cũng chưa có bài báo publish, tuy nhiên việc này thì không mang tính bắt buộc vì khi ta xem những khảo sát thì số người có việc thì số lượng người không có bài báo publish cũng chiếm hơn 50%, tuy rằng không chiếm phần lớn như những người chưa có việc

--------

In [176]:
df = pandas.read_csv('./DataScienceJobsSalaries.csv')

In [177]:
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021e,EN,FT,Data Science Consultant,54000,EUR,64369,DE,50,DE,L
1,2020,SE,FT,Data Scientist,60000,EUR,68428,GR,100,US,L
2,2021e,EX,FT,Head of Data Science,85000,USD,85000,RU,0,RU,M
3,2021e,EX,FT,Head of Data,230000,USD,230000,RU,50,RU,L
4,2021e,EN,FT,Machine Learning Engineer,125000,USD,125000,US,100,US,S
...,...,...,...,...,...,...,...,...,...,...,...
240,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
241,2021e,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
242,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
243,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [186]:
import statsmodels.stats.weightstats as ws

In [203]:
Company_size_L = df[df.company_size == 'L']
Company_size_M = df[df.company_size == 'M']
Company_size_S = df[df.company_size == 'S']
Company_size_L.shape,Company_size_M.shape,Company_size_S.shape

((132, 11), (55, 11), (58, 11))

Kiểm định mức lương ở các công ti

Giả thuyết:

H0: mức lương chi trả ở các công ti có quy mô khác nhau thì khác nhau

H1: các công ti có quy mô khác nhau vẫn có mức chi trả giống nhau

mức ý nghĩa: $\alpha$ =  0.05

số mẫu tại các tập đều lớn hơn 30, nên tả sử dụng z-test

In [214]:
_,pvalue_1  = ws.ztest(Company_size_L.salary_in_usd,Company_size_M.salary_in_usd,alternative='two-sided')
_,pvalue_2  = ws.ztest(Company_size_L.salary_in_usd,Company_size_S.salary_in_usd,alternative='two-sided')
_,pvalue_3  = ws.ztest(Company_size_S.salary_in_usd,Company_size_M.salary_in_usd,alternative='two-sided')

print(pvalue_1,pvalue_2,pvalue_3)

0.033573343820807934 0.012946547257140789 0.7570271963036201


Nhận xét:
- p_value < $\alpha$ tại các trường hợp công ti có quy mô lớn (L), và p_value > $\alpha$ tại trường hợp công ti nhỏ (S) và trung bình (M), khẳng định có sự khác nhau giữa công ti có quy mô lớn với các quy mô khác, trong khi không có sự khác biệt giữa công ti nhỏ và vừa

Kết luận:
- Kết quả cũng không nằm ngoài dự đoán của nhiều người, khi các công ti có quy mô lớn có phúc lợi nhiều hơn các công ti khác, tuy nhiên có một điểm đáng chú ý mà cũng có thể coi là điểm tốt khi không có nhiều sự khác biệt giữa công ti nhỏ và vừa, giúp cho những người đang tìm việc có tâm lý thoải mãi hơn khi chọn lựa giữa các công ti tầm trung và nhỏ

-----------

In [209]:
df.remote_ratio.unique()

array([ 50, 100,   0], dtype=int64)

In [211]:
remote_0 = df[df.remote_ratio == 0]
remote_50 = df[df.remote_ratio == 50]
remote_100 = df[df.remote_ratio == 100]

remote_0.shape,remote_50.shape,remote_100.shape

((40, 11), (71, 11), (134, 11))

Kiểm định mức lương ở các phương thức làm việc khác nhau

Giả thuyết:

H0: có sự khác biệt giữa mức lương ở các hình thức làm việc

H1: hình thức làm việc không ảnh hưởng đến lương

mức ý nghĩa: $\alpha$ =  0.05

số mẫu tại các tập đều lớn hơn 30, nên tả sử dụng z-test

In [224]:
_,pvalue_1  = ws.ztest(remote_0.salary_in_usd,remote_50.salary_in_usd,alternative='two-sided')
_,pvalue_2  = ws.ztest(remote_0.salary_in_usd,remote_100.salary_in_usd,alternative='two-sided')
_,pvalue_3  = ws.ztest(remote_50.salary_in_usd,remote_100.salary_in_usd,alternative='two-sided')

print(pvalue_1,pvalue_2,pvalue_3)

0.7032689939230206 0.061142764809707095 0.0030102779248956414


nhận xét:

- p_value < $\alpha$ tại các trường hợp làm việc remote hoàn toàn (100%), và p_value > $\alpha$ tại trường hợp không làm việc remote (0%) và làm việc kết hợp remote (50%). Khẳng định rằng mức lương sẽ có một sự khác biệt khi ta muốn làm việc remote hoàn toàn

Kết luận:

- Kết quả đưa ra không quá ngạc nhiên khi môi trường remote hoàn toàn có mức lương khác hẳn các hình thức làm việc truyền thống. 
- Tuy thế thì cũng phản ánh được một sự cần cân nhắc cho người lao động khi lựa chọn hình thức remote hoàn toàn khi đây có thể coi là 'xu hướng' làm việc của nhiều người lao động hiện tại trong ngành công nghệ